In [ ]:
import os
import cv2
import json
import torch
import numpy as np
from torch import nn
import torchvision
from scipy.io import loadmat
from torchvision import transforms
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import precision_score, f1_score
from cluster import RLOMTFAGCluster

from sklearn.model_selection import train_test_split

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128
SEED = 42
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
# torch.manual_seed(SEED)
# target = torch.utils.data.Subset(trainset, range(0, 10000))
# torch.manual_seed(SEED)
# train = torch.utils.data.Subset(trainset, range(10000, 50000))

torch.manual_seed(SEED)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
# torch.manual_seed(SEED)
# targetloader = torch.utils.data.DataLoader(target, batch_size=batch_size,
#                                           shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Convert the RGB images to an nd-array,
# the values are the gray scale,
# each column is an image
def convert_to_ndarray(loader):
    images = []
    torch.manual_seed(SEED)
    for batch_idx, (inputs, targets) in enumerate(loader):
        for i in range(len(targets)):
            img = inputs[i]
            img = img * 0.5 + 0.5
            img = img.numpy()
            img = np.transpose(img, (1, 2, 0))
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            images.append(img.flatten())
    images = np.array(images).T
    return images


# target_images = convert_to_ndarray(targetloader)
train_images = convert_to_ndarray(trainloader)
# test_images = convert_to_ndarray(testloader)

# train_images.shape, test_images.shape

target_images = train_images[:, 0:256]
param = [1e-6, 1e-4, 1e-5, 16, 10]
payload = {'X': target_images,
           'k': 10,
           'param': param,
           'limiter': 100,
           'epsilon': 1e-10,
           'distance_metric': 'cosine'}

rlomtfag_cls = RLOMTFAGCluster()
rlomtfag_cls.fit(payload)

In [ ]:
class SE_Block(nn.Module):
    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1, 1)
        return x * y.expand_as(x)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class SENet(nn.Module):
    def __init__(self):
        super(SENet, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 6, 5), nn.ReLU(),  # 32, 32, 32
            nn.MaxPool2d(kernel_size=2, stride=2),  # 32, 16, 16
            nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),  # 64, 12, 12
            SE_Block(c=16, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 64, 6, 6
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
            nn.Linear(120, 84), nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.net(x)
        return x


class EnhancedNet(nn.Module):
    def __init__(self):
        super(EnhancedNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.se = SE_Block(c=16, r=4)

    def forward(self, x):
        X = []
        for i in range(len(x)):
            img = x[i]
            img = img * 0.5 + 0.5
            img = img.detach().cpu().numpy()
            img = np.transpose(img, (1, 2, 0))
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            X.append(img.flatten())
        X = np.array(X).T
        cls_fea = rlomtfag_cls.project(X).T
        cls_fea = torch.from_numpy(cls_fea).float()
        cls_fea = cls_fea.to(device)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(self.se(F.relu(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        cls_fea = torch.sigmoid(cls_fea)
        x = torch.add(x, cls_fea)
        return x

In [ ]:
net = EnhancedNet().to(device)

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_num, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        # 将梯度初始化为0，pytorch中的梯度会自动累加，因此每次都要初始化
        optimizer.zero_grad()
        result = model(data)
        # 计算损失
        loss = F.cross_entropy(result, label)
        # Backward
        loss.backward()
        # Update 参数
        optimizer.step()
        if (batch_num + 1) % 60 == 0:
            print(
                f'Epoch: {epoch}\tBatch: {batch_num + 1}\tLoss: {loss.item():.6f}')


def test_model(model, device, test_loader):
    # 验证模型
    model.eval()
    # loss
    test_loss = 0.0
    pred_all = []
    label_all = []
    labels = list(range(10))
    # 测试过程不需要计算梯度和反向传播
    with torch.no_grad():
        for data, label in test_loader:
            label_all.extend(label)
            data, label = data.to(device), label.to(device)
            result = model(data)
            # 计算test损失
            test_loss += F.cross_entropy(result, label).item()
            # 找到最大值下标
            pred = result.argmax(dim=1)
            pred_all.extend(pred.cpu().data.numpy())

    test_loss /= len(test_loader.dataset)

    confmat = confusion_matrix(torch.tensor(label_all), torch.tensor(pred_all))
    print(f'{confmat}')

    pred_all = label_binarize(pred_all, classes=labels)
    label_all = label_binarize(label_all, classes=labels)
    precision = precision_score(label_all, pred_all, average='macro')
    recall = recall_score(label_all, pred_all, average='macro')
    f1 = f1_score(label_all, pred_all, average='macro')
    acc = accuracy_score(label_all, pred_all)

    print("\nPrecision Score: ", precision)
    print("Recall Score: ", recall)
    print("F1 Score: ", f1)
    print("Accuracy Score: ", acc)
    print(f'\nTest Average Loss: {test_loss:.6f}')
    print(
        f"\nAUC: {roc_auc_score(label_all, pred_all, average='macro', multi_class='ovo', labels=labels):.4f}")

In [ ]:
for epoch in range(1, 30 + 1):
    train_model(net, device, trainloader, optimizer, epoch)
    test_model(net, device, testloader)